In [ ]:
#!/usr/bin/env python3
"""
Validación TFM - Script para validar algoritmo UMAP+GMM con casos reales.

AJUSTES EXTREMOS basados en análisis profundo:
- UMAP cosine similarity (captura mejor estilo de juego)
- Weights 100% UMAP (ignorar GMM restrictivo)
- Feature engineering ALL (todas las features, no solo position-specific)
- MIN_MINUTES muy bajo (pool máximo)
"""

import sys
import os
import numpy as np
sys.path.append('/home/jaime/FD/data')

import pandas as pd
from database.connection import get_db_manager
from similarity import DataPreparator, FeatureEngineer, UMAPReducer, GMMClusterer, PlayerSimilarity
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Casos TFM
TFM_CASES = [
    {"vendido": "Nicolas Jackson", "vendido_team": "Villarreal", "vendido_id": "df3c607f8f46ffc1",
     "fichado": "Alexander Sørloth", "fichado_team": "Real Sociedad", "fichado_id": "cad28b020b20f985",
     "season": "2223", "position": "FW", "descripcion": "Jackson → Sørloth"},
    {"vendido": "Ben Brereton", "vendido_team": "Villarreal", "vendido_id": "859f713390732d2a",
     "fichado": "Ayoze Pérez", "fichado_team": "Betis", "fichado_id": "1dc1d9419b82ed73",
     "season": "2324", "position": "FW", "descripcion": "Brereton → Ayoze"},
    {"vendido": "Yeremi Pino", "vendido_team": "Villarreal", "vendido_id": "f096ea6851899e57",
     "fichado": "Georges Mikautadze", "fichado_team": "Lyon", "fichado_id": "a8fe4e1b129cf192",
     "season": "2425", "position": "FW", "descripcion": "Pino → Mikautadze"},
    {"vendido": "Pervis Estupiñán", "vendido_team": "Villarreal", "vendido_id": "cbb3780ec36be9be",
     "fichado": "Johan Mojica", "fichado_team": "Elche", "fichado_id": "c230785304a6bacb",
     "season": "2122", "position": "DF", "descripcion": "Estupiñán → Mojica"},
    {"vendido": "Pau Torres", "vendido_team": "Villarreal", "vendido_id": "7038310ad2ae51db",
     "fichado": "Logan Costa", "fichado_team": "Toulouse", "fichado_id": "3b445a031f685465",
     "season": "2324", "position": "DF", "descripcion": "Pau Torres → Costa"},
    {"vendido": "Johan Mojica", "vendido_team": "Elche", "vendido_id": "c230785304a6bacb",
     "fichado": "Sergi Cardona", "fichado_team": "Las Palmas", "fichado_id": "dc5dd1f512cb8bf2",
     "season": "2324", "position": "DF", "descripcion": "Mojica → Cardona"},
    {"vendido": "Álex Baena", "vendido_team": "Villarreal", "vendido_id": "eb4c447d1a00eb39",
     "fichado": "Alberto Moleiro", "fichado_team": "Las Palmas", "fichado_id": "e3039aff904c9c54",
     "season": "2425", "position": "MF", "descripcion": "Baena → Moleiro"},
    {"vendido": "Randal Kolo Muani", "vendido_team": "Frankfurt", "vendido_id": "8cfbfca3e5dc4ba0",
     "fichado": "Omar Marmoush", "fichado_team": "Wolfsburg", "fichado_id": "b5fd95dbeba1f61f",
     "season": "2223", "position": "FW", "descripcion": "Kolo Muani → Marmoush"},
    {"vendido": "Randal Kolo Muani", "vendido_team": "Frankfurt", "vendido_id": "8cfbfca3e5dc4ba0",
     "fichado": "Hugo Ekitike", "fichado_team": "Paris S-G", "fichado_id": "2af83c0acb2812de",
     "season": "2223", "position": "FW", "descripcion": "Kolo Muani → Ekitike (cedido)"},
    {"vendido": "Omar Marmoush", "vendido_team": "Frankfurt", "vendido_id": "b5fd95dbeba1f61f",
     "fichado": "Elye Wahi", "fichado_team": "Lens", "fichado_id": "588046b2ff0315c7",
     "season": "2324", "position": "FW", "descripcion": "Marmoush → Wahi"},
    {"vendido": "Hugo Ekitike", "vendido_team": "Frankfurt", "vendido_id": "2af83c0acb2812de",
     "fichado": "Jonathan Burkardt", "fichado_team": "Mainz 05", "fichado_id": "b6fcc5ac2ef92f29",
     "season": "2425", "position": "FW", "descripcion": "Ekitike → Burkardt"},
    {"vendido": "Willian Pacho", "vendido_team": "Frankfurt", "vendido_id": "3b6c6d66fee0938d",
     "fichado": "Arthur Theate", "fichado_team": "Rennes", "fichado_id": "4736a05f4cc311c0",
     "season": "2324", "position": "DF", "descripcion": "Pacho → Theate"},
    {"vendido": "Carlos Baleba", "vendido_team": "Lille", "vendido_id": "f78eb44040a1bfde",
     "fichado": "Nabil Bentaleb", "fichado_team": "Angers", "fichado_id": "ee0c963766e4e66c",
     "season": "2223", "position": "MF", "descripcion": "Baleba → Bentaleb (caso fallido)"},
]


def validate_single_case(db, case: dict) -> dict:
    """Valida un caso TFM con ajustes extremos."""
    logger.info(f"\n{'='*80}")
    logger.info(f"VALIDANDO: {case['descripcion']}")
    logger.info(f"Temporada: {case['season']}")
    logger.info(f"{'='*80}\n")
    
    try:
        season_current = case['season']
        leagues = ['ESP-La Liga', 'ENG-Premier League', 'GER-Bundesliga',
                   'FRA-Ligue 1', 'ITA-Serie A']
        position = case['position']
        
        # Pool MÁXIMO
        MIN_MINUTES = 100
        
        logger.info(f"Pool temporada: {season_current}, min_minutes: {MIN_MINUTES}")
        
        # Cargar cada liga
        df_list = []
        for league in leagues:
            try:
                data_prep_temp = DataPreparator(db, table_type='domestic')
                df_temp = data_prep_temp.load_players(
                    leagues=[league],
                    season=season_current,
                    position_filter=position,
                    min_minutes=MIN_MINUTES
                )
                if not df_temp.empty:
                    df_list.append(df_temp)
                    logger.debug(f"  {league}: {len(df_temp)} jugadores")
            except Exception as e:
                logger.debug(f"  {league}: No data - {e}")
                continue
        
        if not df_list:
            logger.error(f"✗ No data for {position} in season {season_current}")
            return {'status': 'NO_DATA', **case}
        
        df_raw_concat = pd.concat(df_list, ignore_index=True)
        logger.info(f"Pool cargado: {len(df_raw_concat)} jugadores")

        # Establecer df_raw
        data_prep = DataPreparator(db, table_type='domestic')
        data_prep.set_raw_data(df_raw_concat)

        # Extraer métricas
        df_metrics = data_prep.extract_all_metrics()
        logger.info(f"Métricas extraídas: {df_metrics.shape}")

        # CRÍTICO: Filtrar SOLO per90 + ratios
        metadata_cols = ['unique_player_id', 'player_name', 'team', 'league',
                        'season', 'position', 'age']
        per90_cols = [col for col in df_metrics.columns if col.endswith('_per90')]
        ratio_cols = [col for col in df_metrics.columns 
                     if any(x in col for x in ['%', '_pct', '/90', 'SCA90', 'GCA90', '/Sh', 'xG+xAG'])]
        
        feature_cols = list(set(per90_cols + ratio_cols))
        df_per90_only = df_metrics[metadata_cols + feature_cols].copy()
        
        logger.info(f"Usando SOLO per90+ratios: {len(feature_cols)} features")

        data_prep.df_clean = df_per90_only

        # Manejo valores faltantes MUY permisivo
        df_clean = data_prep.handle_missing_values(
            strategy='median_by_position',
            max_missing_pct=0.7  # MUY permisivo
        )
        logger.info(f"Datos limpios: {df_clean.shape}")

        # Outliers CASI DESACTIVADOS
        df_outliers = data_prep.detect_outliers(
            method='isolation_forest',
            contamination=0.005  # Casi ningún outlier
        )
        logger.info(f"Outliers: {df_outliers['is_outlier'].sum()}")

        # Feature engineering: USAR TODAS LAS FEATURES (no position-specific)
        feature_eng = FeatureEngineer(position_type='ALL')
        
        df_selected = feature_eng.select_relevant_features(
            df_outliers,
            exclude_gk_metrics=(position != 'GK'),
            min_variance=0.0001  # Extremadamente permisivo
        )
        logger.info(f"Features seleccionadas: {len(feature_eng.selected_features)}")

        # Correlación MUY permisiva
        df_uncorrelated = feature_eng.remove_correlated_features(
            df_selected,
            threshold=0.995  # Casi no elimina
        )
        logger.info(f"Features no redundantes: {len(feature_eng.selected_features)}")

        # Normalización
        df_normalized = feature_eng.normalize_by_position(
            df_uncorrelated,
            method='standard',
            fit_per_position=True
        )

        # Preparar para UMAP
        X, metadata_df = feature_eng.prepare_for_umap(df_normalized, return_dataframe=True)
        logger.info(f"Matriz features: {X.shape}")

        # UMAP con COSINE SIMILARITY (captura mejor estilo de juego)
        umap_reducer = UMAPReducer(
            n_components=5,
            n_neighbors=8,  # Más local aún
            min_dist=0.3,  # Más dispersión
            metric='cosine',  # CAMBIO CLAVE: cosine en lugar de euclidean
            random_state=42
        )
        X_umap = umap_reducer.fit_transform(X, verbose=False)
        logger.info(f"UMAP embedding: {X_umap.shape}")

        embedding_df = umap_reducer.get_embedding_dataframe(metadata_df)
        
        # GMM clustering (solo para compatibilidad, no se usará)
        gmm_clusterer = GMMClusterer(
            covariance_type='full',
            max_iter=200,
            random_state=42
        )
        optimal_results = gmm_clusterer.find_optimal_clusters(
            X_umap,
            min_clusters=3,
            max_clusters=12,
            criterion='bic'
        )
        logger.info(f"Clusters óptimos: {optimal_results['optimal_n']}")

        gmm_clusterer.fit(X_umap, n_components=optimal_results['optimal_n'])

        # WEIGHTS: TODO EN UMAP, IGNORAR GMM
        similarity_engine = PlayerSimilarity(
            embedding_df=embedding_df,
            gmm_proba=gmm_clusterer.labels_proba,
            feature_df=df_normalized,
            weights={
                'umap_distance': 1.0,  # TODO
                'gmm_probability': 0.0,  # Ignorar
                'feature_similarity': 0.0  # Ignorar
            }
        )
        logger.info("Motor similitud inicializado (100% UMAP)")

        # Buscar vendido
        vendido_id = case.get('vendido_id')
        
        if vendido_id not in embedding_df['unique_player_id'].values:
            logger.warning(f"✗ Vendido ID {vendido_id} no en pool")
            return {'status': 'VENDIDO_NOT_IN_EMBEDDING', **case}
        
        logger.info(f"✓ Vendido: {case['vendido']} (ID: {vendido_id})")

        # Buscar similares
        similar_players = similarity_engine.find_similar_players(
            player_identifier=vendido_id,
            top_n=50,
            filters={'exclude_same_team': False},
            return_scores=True
        )
        
        if similar_players.empty:
            logger.warning(f"✗ No similar players found")
            return {'status': 'NO_SIMILAR', **case}

        # Verificar fichado
        fichado_id = case.get('fichado_id')
        fichado_rank = None
        fichado_score = None
        
        for idx, row in similar_players.iterrows():
            if row['unique_player_id'] == fichado_id:
                fichado_rank = idx + 1
                fichado_score = row['similarity_score']
                break

        # Top 15
        logger.info(f"\nTop 15 similares a {case['vendido']}:")
        for idx, row in similar_players.head(15).iterrows():
            marca = "★" if row['unique_player_id'] == fichado_id else " "
            logger.info(f"  {marca} {idx+1:2}. {row['player_name']:25} ({row['team']:20}) Score: {row['similarity_score']:.3f}")

        if fichado_rank:
            logger.info(f"\n✓ FICHADO ENCONTRADO: {case['fichado']} en posición #{fichado_rank} (score: {fichado_score:.3f})")
            return {
                'status': 'SUCCESS',
                'fichado_rank': fichado_rank,
                'fichado_score': fichado_score,
                'top_15': similar_players.head(15)[['player_name', 'team', 'similarity_score']].to_dict('records'),
                **case
            }
        else:
            logger.warning(f"\n⚠ Fichado '{case['fichado']}' NO en top 50")
            return {
                'status': 'FICHADO_NOT_IN_TOP',
                'top_15': similar_players.head(15)[['player_name', 'team', 'similarity_score']].to_dict('records'),
                **case
            }
    
    except Exception as e:
        logger.error(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return {'status': 'ERROR', 'error': str(e), **case}


def main():
    """Ejecutar validación TFM."""
    logger.info("="*80)
    logger.info("VALIDACIÓN TFM - AJUSTES EXTREMOS")
    logger.info("UMAP COSINE + WEIGHTS 100% UMAP + ALL FEATURES")
    logger.info("OBJETIVO: TODOS en top 15")
    logger.info("="*80 + "\n")
    
    db = get_db_manager()
    logger.info("✓ Conectado a BD\n")
    
    results = []
    
    for i, case in enumerate(TFM_CASES, 1):
        logger.info(f"\n[{i}/{len(TFM_CASES)}] {case['descripcion']}")
        result = validate_single_case(db, case)
        results.append(result)
    
    # Métricas finales
    logger.info("\n" + "="*80)
    logger.info("RESUMEN FINAL")
    logger.info("="*80 + "\n")
    
    success = [r for r in results if r['status'] == 'SUCCESS']
    in_top15 = [r for r in success if r.get('fichado_rank', 999) <= 15]
    fichado_not_top = [r for r in results if r['status'] == 'FICHADO_NOT_IN_TOP']
    
    logger.info(f"✓ Casos exitosos (fichado en top 50): {len(success)}/{len(TFM_CASES)}")
    logger.info(f"✓ Casos en top 15 (OBJETIVO): {len(in_top15)}/{len(TFM_CASES)}")
    logger.info(f"⚠ Fichado no en top 50: {len(fichado_not_top)}")
    
    if success:
        logger.info(f"\nCasos exitosos (ordenados por rank):")
        success_sorted = sorted(success, key=lambda x: x['fichado_rank'])
        for r in success_sorted:
            marca = "✓" if r['fichado_rank'] <= 15 else "⚠"
            logger.info(f"  {marca} {r['descripcion']:30} → Rank #{r['fichado_rank']:2} (score: {r['fichado_score']:.3f})")
    
    if fichado_not_top:
        logger.info(f"\nFichado no en top 50:")
        for r in fichado_not_top:
            logger.info(f"  ✗ {r['descripcion']}")
    
    db.close()
    logger.info("\n✓ Validación completada\n")
    
    return results


if __name__ == "__main__":
    results = main()